In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

from torch.utils.data import TensorDataset, DataLoader

from llama import Tokenizer
tokenizer = Tokenizer('./tokenizer.model')

#hyperparameters
n_embd = 256
block_size = 128
n_head = 4
n_layer = 4
vocab_size = 32_000
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3

text = open('../data/shakespeare.txt', encoding="utf-8").read()

data = torch.tensor(tokenizer.encode(text, False, False))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

def get_all_data(split):
    
    data = train_data if split == "train" else val_data
    X = torch.stack([data[i:i+block_size] for i in range(len(data) - block_size)])
    y = torch.stack([data[i+1:i+1+block_size] for i in range(len(data) - block_size)])
    X, y = X.to(device), y.to(device)
    return X, y


X, y = get_all_data('train')
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    
class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd)
        )
        
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        x = x + self.ln1(self.sa(x))
        x = x + self.ln2(self.ffwd(x))
        
        return x
    
class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            
            B,T,C = logits.shape
            
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_gen_len):
        
        initial_idx = idx
        
        out = torch.zeros(max_gen_len).to(device)
        
        for i in range(max_gen_len):
            
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            
            probs = F.softmax(logits, dim=-1)
            
            pred = torch.multinomial(probs, 1)
            
            out[i] = pred.item()
            
            idx = torch.cat((idx, pred), dim=-1)
            
        out = torch.cat((initial_idx.squeeze(0), out), dim=-1)
            
        print(tokenizer.decode(out.long().tolist()))
            
    
        
        
    
model = Transformer().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



In [20]:
model.generate(idx=torch.tensor(tokenizer.encode('HERMIONE:', False, False)).unsqueeze(0).to(device), max_gen_len=500)

HERMIONE: if I came to bread in a bagary these thus, like the nay'd
We are place else to be at the skullus Aufid, for a duke fits; soft for this attempt and sorrow'll bring sheed with you, mark me with him: hath carried of all.
IUS:

Lest!
LUCIO:
say know
Well with him run and with me with us all the like an descent. Even at Baynable to drows bred
The crafted and in.
I like a period
IUS:

I as's adversaries a vexret ANNE:
And brought as well to that ungas!
I will be gone before
Would inclontes! Flybly rascal; 'thrifty. Therefore, and hear a widowards the enmity hope to envious floods--chesess to thy wrong's do it yet! He,
In humble
I' Thou plague is the treasonY: counts me see her dowy attain'd
We may surrenderended HENENI will not the consenting peiness of his coun't,
wholesome rid me
All of living fire:
JUDIO,
I here beats and that men to the stronger to the effect it be distribute
Will you pale?
Preparewell the fond:
And spilt perform him;
Well, where'd him with God, to challenge he

In [19]:
count = 0
epochs = 500

for xb, yb in dataloader:
    
    logits, loss = model(xb, yb)
    
    if count % (epochs // 10) == 0:
        print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    count += 1
    if count == epochs:
        break

tensor(10.5356, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(5.7685, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.9979, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.4395, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.3353, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.2119, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0444, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0662, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0027, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(4.0553, device='cuda:0', grad_fn=<NllLossBackward0>)
